In [1]:
import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms
import numpy as np

## CNN's

ConvNets exploit natural image structure, reduce data in latent space without computationally intractable operations by aggregation.

**Spatial Invariance** - sweep an image by patches to learn representation of each subpatch of image, then aggregate a representation. (Aggregation of subspace).


### 2 Major Criteria to make this suitable for Computer vision:


**Translation Invariance** - Each Region should be initially treated indiscriminantly (we attempt to learn an identical thing about each part of the image initially).

**Locality** - We focus on subregions of the image to begin with, without concern for how these subregions will affect other distant regions until later layers when the information is aggregated.

### Criteria Application in ConvNets

In a ConvNet, we take a weighted function of pixles within a locality. We place the result of this output in the corresponding output subsection, AKA corner aggregation maintains corner status, thus maintaining the concept of spatiality.

Translational Invariance is maintained by keeping the weight filter constant, such that no matter where we are in the image we are maintaining the same "Question" / learning an identical thing about each part of the image initially.

Locality is maintained by having a convolutional filter which is less than the size of the image, thus aggregating information along various localities (subregions). AKA if v is the **kernel** / **filter** (this is an abstraction of a Kernel seen in Kernel Density, which is a weighting function for nearby points / features valuing them by distances, with a Neural Kernel instead valuing them by placement) then v > index i for some i will be 0 (far away features not weighted at all as do not impact local subimage understanding, just as window in KDE is capped by lambda

Locality allows us to shrink our filters below the size of the image and learn local filters. Transitional invariance allows us to use one such filter uniformly across the image indiscriminantly. These together combine to the idea that we can learn a homogeneous feature about each locality, and by aggregating such homogenous questions can learn a valuable latent representation that can be used to classify an image. This methodology is extremely parameter efficient, however, as we only need to learn one weighting function per layer, drastically reducing the combinatorial parameterization of our models to a few hundred

### Why is it called a Convolution?

Convolution is probability joint distribution adds up to something which is = to integral of a = x and b = z-x for across all x. This is equivalent to a sliding window across x in the discrete sense, because the operation in and of itself does not look at joint probability but does weigh possibilities in a convolutional fashion. In the 2 dim sense: x(a,b) * y(z-a, w-b) is a sliding window across the entirety of the [0,z]x[0,w] possibility space, attempting all combinatorial possibilities discretely. That is a rough discrete mapping to a convolution, where we aggregate all joint possibilities to understand the joint probability of an outcome of two variables. Also use one function to scale the other with an AND clause, just as is done in a convoluted multiplication

In practice the mapping of the sliding filter is actually 3 dimensional to account for a third dimension: color in the RGB space

Fundamental Operation: We convolve a weighting filter across an image to obtain an aggregation of several localities

## Convolutions on Images

Operation is far closer to cross correlation (dot product) as aggregating how frequent our valuable features are. If intensity in a certain position is given a higher weight and it is more intense, then we will encode a greater number of logits of information (a valuable feature for classification present there, can map to ranges (37+ = presence of features => cat) This cross correlation is implemented below

In [2]:
def cross_cor(t1: torch.Tensor, kernel: torch.Tensor) -> torch.Tensor:
    """Applies a kernel filter across a tensor, producing an aggregated latent representation"""
    # New shape is how far short of endpoint filter must stop (because it cannot exceed boundaries of image) + 1 for the stopping iteration (if it is sized 2, it still aggregates when 2 away from edge)
    new_shape = t1.shape[0] - kernel.shape[0] + 1, t1.shape[1] - kernel.shape[1] + 1
    # A latent representation of aggregated localities
    latent_representation = torch.rand(new_shape[0], new_shape[1]) # Using image dimensions
    # Iterating and cross correlating
    for i in range(new_shape[0]):
        for j in range(new_shape[1]):
            # Will dot tensors together via multiplication operator - this is the default
            # Obtaining locality, using stride of 1, of sized kernel. Convoluted along
            inter_tensor = t1[i:i+ kernel.shape[0], j:j+kernel.shape[1]]
            # Obtaining cross correlation of locality with kernel (using kernel to weigh sum of information in locality)
            weighted_locality_rep = (inter_tensor * kernel).sum() # Aggregating weighted feature information (reduces latent space rather than just weights)
            latent_representation[i][j] = weighted_locality_rep
    return latent_representation
            
    

In [3]:
cross_cor(torch.randn(4,3), torch.randn(2,2)) # Latent representation maintains aggregated features in localities

tensor([[ 0.0807, -0.7073],
        [ 1.0288, -0.7220],
        [ 2.9811,  0.0174]])

In [4]:
def cross_cor_4d(t1: torch.Tensor, kernel: torch.Tensor) -> torch.Tensor:
    """Applies a kernel filter across a tensor, producing an aggregated latent representation"""
    # New shape is how far short of endpoint filter must stop (because it cannot exceed boundaries of image) + 1 for the stopping iteration (if it is sized 2, it still aggregates when 2 away from edge)
    new_shape = t1.shape[0] - kernel.shape[0] + 1, t1.shape[1] - kernel.shape[1] + 1, t1.shape[2] - kernel.shape[2] + 1, t1.shape[3] - kernel.shape[3] 
    # A latent representation of aggregated localities
    latent_representation = torch.rand(new_shape[0], new_shape[1], new_shape[2], new_shape[3]) # Using image dimensions
    # Iterating and cross correlating
    for i in range(new_shape[0]):
        for j in range(new_shape[1]):
            for k in range(t1.shape[1]):
                # Will dot tensors together via multiplication operator - this is the default
                # Obtaining locality, using stride of 1, of sized kernel. Convoluted along
                inter_tensor = t1[i:i+ kernel.shape[0], j:j+kernel.shape[1], k:k+kernel.shape[2], 0:kernel.shape[3]]
                # Obtaining cross correlation of locality with kernel (using kernel to weigh sum of information in locality)
                weighted_locality_rep = (inter_tensor * kernel).sum() # Aggregating weighted feature information (reduces latent space rather than just weights)
                latent_representation[i][j][k] = weighted_locality_rep
    return latent_representation

In [5]:
# Defining convolutional layer
class Conv2D(torch.nn.Module):
    def __init__(self, kernel_size: tuple):
        super().__init__()
        # These will be a learnable parameter. The NN will learn what it should consider important
        self.weights = torch.nn.Parameter(torch.randn(kernel_size))
        self.bias = torch.nn.Parameter(torch.zeros(1))
        self.weights = torch.nn.init.xavier_uniform_(self.weights)
        
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return torch.nn.functional.relu(cross_cor(X, self.weights) + self.bias)

    
class Flatten(torch.nn.Module):
    """Non-Sequential flatten"""
    
    def __init__(self):
        super().__init__()
    
    def forward(self, inp: torch.Tensor)->torch.Tensor:
        temp = inp.detach().numpy().flatten()
        new_tensor = torch.Tensor(temp)
        new_tensor.requires_grad_ = True
        return new_tensor
        
    
    

In [6]:
class BasicConvNet(torch.nn.Module):
    def __init__(self, *kernel_sizes):
        super().__init__()
        self._modules["layer_1"] = Conv2D(kernel_sizes[0])
        self._modules["layer_2"] = Conv2D(kernel_sizes[1])
        self._modules["layer_3"] = Flatten()
        self._modules["layer_4"] = torch.nn.Linear(100, 10)
    
    def forward(self, X: torch.Tensor)-> torch.Tensor:
        # Applying both convolutions
        X = self._modules["layer_1"](X)
        X = self._modules["layer_2"](X)
        X = self._modules["layer_3"](X)
        X = self._modules["layer_4"](X)
        return X

In [7]:
import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms

In [8]:
# Function from previous notebook
# Converting to function for future use, default num_workers is 4 bc CPU threads
def load_fashion_mnist(batch_size: int = 512, num_workers: int = 4):
    data_transform = transforms.ToTensor() # Obtaining data to tensor converter
    
    # Downloading data
    mnist_train = torchvision.datasets.FashionMNIST(root = "../data", train = True, transform = data_transform, download= True)  # Defining fashion MNIST train from torch datasets
    mnist_test = torchvision.datasets.FashionMNIST(root = "../data", train = False, transform = data_transform, download = True)
    
    # Loading data onto an iterator
    train_data_loader = data.DataLoader(mnist_train, batch_size, shuffle = True, num_workers = 4)
    test_data_loader = data.DataLoader(mnist_test, batch_size, shuffle = True, num_workers = 4)
    
    # Returning iterator
    return train_data_loader, test_data_loader 
    

In [9]:
train_loader, test_loader = load_fashion_mnist(256, 4)

/Users/gilpasternak/Library/Python/3.8/lib/python/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
# Applying init to model to initialize all layer weights
def build_model():
    # Kernels are iterating across all three dimensions and all batch sizes simultaneously
    model = BasicConvNet((15,15,1,1), (5,5,1,1))
    trainer = torch.optim.Adam(model.parameters(), lr = 0.05, weight_decay=0.05)
    loss = torch.nn.CrossEntropyLoss()
    return model, trainer, loss

In [11]:
model, trainer, loss = build_model()

In [12]:
# Note: 4D code is merely an outline

In [13]:
model(torch.randn(28,28)) # 2 dimensional ConvNet Functional

tensor([-1114.2125,  -185.5264,  3063.9458,  -371.9173,  -528.3566, -1614.9824,
        -1885.4993,   417.1822,   485.9057,  1479.3510],
       grad_fn=<AddBackward0>)

In [92]:
class ConvNet(torch.nn.Module):
    def __init__(self, *dims):
        # Weigh + aggregate information twice by convolving it with a localized weighting kernel, then map to output space
        # Call to parent constructor initializes modules
        super().__init__()
        self._modules["layer_1"] = torch.nn.Conv2d(1,1, kernel_size = (14,14))
        self._modules["layer_2"] = torch.nn.Conv2d(1,1, kernel_size = (6,6)) 
        # Previous error: thought labels were 10 class probabilities, turns out we want concrete prediction
        self._modules["remaining_network"] = torch.nn.Sequential(torch.nn.Flatten(), torch.nn.Linear(100,10))
        
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """Instance method defining computational mapping to output space"""
        X = torch.nn.functional.relu(self._modules["layer_1"](X))
        X = torch.nn.functional.relu(self._modules["layer_2"](X))
        # Convolving, breaking linearity, flattening, mapping to output space
        X = self._modules["remaining_network"](X)
        # Returning result
        return X
                                                                        

In [74]:
# 1-1+1, 1-1+1, 5-2+1, 5-1+1 stride 1 => output only loses a single column
torch.nn.Conv2d(1,1, kernel_size = (2,1), bias = False)(torch.rand((1,1, 5,5)))

tensor([[[[-0.3638, -0.4770, -0.2514, -0.4221, -0.3489],
          [-0.2718, -0.4054, -0.0716, -0.3238, -0.5226],
          [-0.1506, -0.1039,  0.0073,  0.0438, -0.5119],
          [-0.0015, -0.1958, -0.3769, -0.2271, -0.3880]]]],
       grad_fn=<ThnnConv2DBackward>)

In [75]:
# Checking if convolution can be performed on Fashion MNIST data
for batch, label in train_loader:
    convoluted = torch.nn.Conv2d(1,1, kernel_size = (19,19), bias = False)(batch)
    print(convoluted)
   

tensor([[[[ 0.1594,  0.2110,  0.2655,  ...,  0.1605,  0.0978,  0.1690],
          [ 0.1676,  0.0988,  0.2400,  ...,  0.1237,  0.0649,  0.1631],
          [ 0.1030,  0.0822,  0.1478,  ...,  0.1349,  0.0788,  0.0356],
          ...,
          [ 0.1853,  0.1556,  0.2444,  ...,  0.2762,  0.2327,  0.0040],
          [ 0.1881,  0.1923,  0.2664,  ...,  0.2801,  0.2087,  0.0205],
          [ 0.1407,  0.2165,  0.2673,  ...,  0.2766,  0.1915,  0.0643]]],


        [[[ 0.4205,  0.4833,  0.5010,  ...,  0.3687,  0.2587,  0.1515],
          [ 0.4999,  0.4247,  0.4325,  ...,  0.3222,  0.2122,  0.0894],
          [ 0.4423,  0.4343,  0.4782,  ...,  0.2855,  0.2647,  0.1197],
          ...,
          [ 0.4311,  0.4907,  0.4523,  ...,  0.2746,  0.3708,  0.2042],
          [ 0.4321,  0.5135,  0.4821,  ...,  0.2834,  0.3927,  0.1988],
          [ 0.3716,  0.4696,  0.4604,  ...,  0.3324,  0.3540,  0.1984]]],


        [[[ 0.1804,  0.2271,  0.0590,  ...,  0.2087,  0.0955,  0.0945],
          [ 0.1931,  0.135

tensor([[[[-0.0197, -0.0200,  0.0265,  ..., -0.0100,  0.0956,  0.0662],
          [-0.0096, -0.0052,  0.0169,  ...,  0.0448,  0.0978,  0.0442],
          [ 0.0095,  0.0281,  0.0027,  ...,  0.1461,  0.0604,  0.1115],
          ...,
          [ 0.0114,  0.1327,  0.1384,  ...,  0.0614, -0.0366, -0.0924],
          [ 0.0142,  0.1403,  0.0980,  ..., -0.0789,  0.0800,  0.0678],
          [ 0.0619,  0.1078,  0.0672,  ..., -0.1058,  0.1066,  0.0749]]],


        [[[ 0.4129,  0.3627,  0.2571,  ...,  0.0770,  0.1097,  0.1453],
          [ 0.1853,  0.1232, -0.1051,  ..., -0.0588, -0.0259,  0.0865],
          [-0.0119, -0.0088, -0.0437,  ..., -0.1689, -0.2151, -0.1490],
          ...,
          [-0.0909, -0.1400, -0.1488,  ..., -0.1159, -0.1527, -0.2495],
          [-0.1093, -0.1480, -0.2177,  ..., -0.0824,  0.0669, -0.0262],
          [-0.0178, -0.0444, -0.1566,  ..., -0.0442, -0.0088, -0.0104]]],


        [[[ 0.1034, -0.0008, -0.0178,  ..., -0.1415, -0.1521, -0.2869],
          [-0.0636, -0.039

tensor([[[[ 0.5948,  0.4311,  0.2466,  ...,  0.0203, -0.2299, -0.2398],
          [ 0.7358,  0.6083,  0.3903,  ...,  0.0411, -0.2103, -0.3100],
          [ 0.6475,  0.5652,  0.3442,  ...,  0.0444, -0.1625, -0.3389],
          ...,
          [ 0.4996,  0.5432,  0.3769,  ...,  0.0892,  0.0049, -0.2726],
          [ 0.5077,  0.5541,  0.4107,  ...,  0.1278,  0.0466, -0.2863],
          [ 0.5065,  0.6114,  0.4275,  ...,  0.1884,  0.1656, -0.2399]]],


        [[[ 0.6599,  0.6394,  0.4382,  ..., -0.0919, -0.0271,  0.0573],
          [ 0.6889,  0.6347,  0.4956,  ..., -0.0799, -0.0429,  0.0182],
          [ 0.6744,  0.5994,  0.5285,  ..., -0.0805, -0.0730,  0.0069],
          ...,
          [ 0.5039,  0.6435,  0.4955,  ..., -0.0211, -0.0636,  0.1556],
          [ 0.5011,  0.6635,  0.4949,  ..., -0.0071, -0.0589,  0.1604],
          [ 0.5063,  0.6673,  0.4961,  ..., -0.0052, -0.0466,  0.1644]]],


        [[[ 0.2289,  0.4438,  0.1982,  ...,  0.2451,  0.2146,  0.1985],
          [ 0.2833,  0.396

tensor([[[[ 0.0201,  0.0804,  0.0445,  ...,  0.0191,  0.0400,  0.0417],
          [ 0.0525,  0.1272,  0.0802,  ...,  0.0584, -0.0098, -0.0270],
          [ 0.1080,  0.1512,  0.0699,  ...,  0.0346,  0.0012, -0.0251],
          ...,
          [ 0.0496, -0.0061,  0.1244,  ...,  0.1345,  0.0496,  0.1721],
          [ 0.0709,  0.0599,  0.1884,  ...,  0.0414, -0.0359,  0.0959],
          [-0.0201, -0.0073,  0.1044,  ..., -0.0357, -0.0780,  0.0675]]],


        [[[-0.0447, -0.0677, -0.1118,  ...,  0.0256, -0.0904,  0.0336],
          [-0.0256, -0.0303,  0.0035,  ..., -0.0604, -0.0311,  0.0985],
          [ 0.0058,  0.0239,  0.1227,  ..., -0.0177, -0.0588,  0.0979],
          ...,
          [-0.0647, -0.0073,  0.1706,  ...,  0.0663, -0.0144,  0.1003],
          [-0.0384, -0.0268,  0.1261,  ...,  0.0369, -0.0312,  0.0879],
          [-0.0663, -0.0995,  0.0757,  ...,  0.1190, -0.0182,  0.0446]]],


        [[[ 0.0265, -0.1851, -0.1038,  ...,  0.0190, -0.0006,  0.0136],
          [-0.0139, -0.145

tensor([[[[ 0.0687,  0.1074,  0.0848,  ...,  0.0541, -0.0263, -0.1040],
          [ 0.2891,  0.1173,  0.1832,  ...,  0.1894,  0.0749, -0.0612],
          [ 0.3072,  0.1313,  0.1825,  ...,  0.1736,  0.1405, -0.0312],
          ...,
          [ 0.0907,  0.1148,  0.0535,  ...,  0.0239,  0.1102,  0.0956],
          [ 0.1824,  0.0928,  0.0099,  ...,  0.0453,  0.0840,  0.0348],
          [ 0.1576,  0.1030, -0.0861,  ...,  0.0096,  0.1009,  0.0324]]],


        [[[ 0.2183,  0.2585,  0.1169,  ...,  0.1393, -0.0329, -0.1123],
          [ 0.2344,  0.2828,  0.2961,  ...,  0.0490,  0.0123, -0.0478],
          [ 0.2474,  0.1489,  0.2405,  ...,  0.0875,  0.1240,  0.0289],
          ...,
          [ 0.2119,  0.1565,  0.1061,  ...,  0.1092,  0.2729,  0.2259],
          [ 0.1955,  0.1744,  0.1044,  ...,  0.2124,  0.3796,  0.2278],
          [ 0.2260,  0.1284,  0.0910,  ...,  0.3923,  0.3864,  0.2351]]],


        [[[ 0.0542,  0.0220,  0.0565,  ...,  0.0242, -0.0122, -0.0527],
          [ 0.0323,  0.066

...

tensor([[[[ 1.1889e-01, -8.1413e-02, -1.4746e-01,  ..., -1.1453e-01,
            4.1236e-02,  2.8748e-02],
          [ 1.7329e-01,  7.3204e-02, -3.5570e-03,  ..., -1.7172e-01,
            5.1152e-02,  2.5733e-02],
          [ 2.4972e-01,  1.7752e-01,  1.0924e-01,  ..., -1.3685e-01,
            2.5953e-02,  1.2244e-01],
          ...,
          [ 2.6643e-01,  1.2644e-01,  1.5384e-01,  ...,  7.9523e-02,
           -5.7928e-03,  9.5691e-02],
          [ 1.9372e-01,  7.4964e-02,  1.8777e-01,  ...,  2.6242e-02,
           -2.8873e-02,  4.5719e-02],
          [ 1.7755e-01, -1.2835e-02,  1.7128e-01,  ...,  2.2314e-02,
           -5.5292e-02, -1.0403e-02]]],


        [[[ 4.1555e-03,  3.3406e-02, -6.7169e-03,  ...,  5.8187e-03,
           -2.3129e-01, -7.5177e-02],
          [ 4.2655e-02,  1.8405e-02, -2.9710e-03,  ..., -5.3395e-02,
           -1.3308e-01, -3.1466e-02],
          [ 2.8564e-02,  7.1010e-02,  5.5437e-02,  ..., -1.3554e-01,
           -8.5625e-02, -1.7084e-01],
          ...,
   

tensor([[[[ 0.4243,  0.4697,  0.5111,  ...,  0.4706,  0.3773,  0.3280],
          [ 0.3891,  0.3687,  0.3167,  ...,  0.5165,  0.4361,  0.3039],
          [ 0.4142,  0.3120,  0.2832,  ...,  0.5455,  0.4268,  0.2875],
          ...,
          [ 0.3814,  0.2646,  0.3019,  ...,  0.6488,  0.6402,  0.3818],
          [ 0.3620,  0.2607,  0.2902,  ...,  0.6516,  0.6497,  0.4075],
          [ 0.2649,  0.0711,  0.1501,  ...,  0.5939,  0.5385,  0.3832]]],


        [[[-0.0304, -0.0457, -0.0873,  ..., -0.0796, -0.1171, -0.0251],
          [-0.0455, -0.1164, -0.0330,  ..., -0.0108, -0.0469, -0.0032],
          [ 0.0132, -0.0146,  0.0690,  ...,  0.0949,  0.0655,  0.1744],
          ...,
          [ 0.1030, -0.0142, -0.0135,  ...,  0.2034,  0.2820,  0.3516],
          [ 0.2120,  0.1991,  0.2109,  ...,  0.2729,  0.2690,  0.1217],
          [-0.0387, -0.0504,  0.1534,  ...,  0.2013,  0.1050, -0.0385]]],


        [[[ 0.0097,  0.1046,  0.1615,  ...,  0.1764,  0.0236,  0.1304],
          [ 0.0469,  0.103

tensor([[[[-0.0346, -0.1565, -0.2884,  ..., -0.3105, -0.3216, -0.4549],
          [-0.1091, -0.1761, -0.1550,  ..., -0.3077, -0.3839, -0.3441],
          [-0.2037, -0.0934, -0.0215,  ..., -0.3210, -0.1874, -0.2022],
          ...,
          [-0.1319, -0.0953, -0.1641,  ..., -0.4126, -0.2663, -0.2704],
          [-0.2018, -0.3074, -0.3809,  ..., -0.3648, -0.3308, -0.4137],
          [-0.2368, -0.2301, -0.2215,  ..., -0.2403, -0.2391, -0.4058]]],


        [[[ 0.1808, -0.0629, -0.0559,  ..., -0.3225, -0.3476, -0.4402],
          [ 0.0363, -0.0861, -0.1912,  ..., -0.3119, -0.4589, -0.4424],
          [ 0.0374, -0.0426, -0.1048,  ..., -0.3047, -0.3045, -0.4326],
          ...,
          [-0.4485, -0.2410, -0.1648,  ..., -0.4117, -0.4616, -0.4493],
          [-0.2356, -0.1438, -0.2523,  ..., -0.4525, -0.5210, -0.4754],
          [-0.1031, -0.1225, -0.2353,  ..., -0.3537, -0.3196, -0.4092]]],


        [[[-0.1117, -0.2025, -0.1495,  ..., -0.3528, -0.3924, -0.2953],
          [-0.2301, -0.177

tensor([[[[ 0.1972,  0.1847,  0.0461,  ...,  0.1129,  0.1856,  0.2084],
          [ 0.1537,  0.2488,  0.0677,  ...,  0.1710,  0.1708,  0.1551],
          [ 0.2211,  0.1738,  0.1741,  ...,  0.1086,  0.1524,  0.1985],
          ...,
          [ 0.2216,  0.2068,  0.1224,  ...,  0.0385,  0.0384,  0.1969],
          [ 0.0995,  0.2265,  0.0965,  ...,  0.1556,  0.0861,  0.0950],
          [ 0.0679,  0.1947,  0.2059,  ...,  0.1161,  0.0268,  0.1079]]],


        [[[ 0.2800,  0.1880,  0.0928,  ...,  0.0823,  0.2314,  0.2884],
          [ 0.2577,  0.1891,  0.1556,  ...,  0.0827,  0.1944,  0.2706],
          [ 0.2163,  0.1759,  0.1814,  ...,  0.0610,  0.1809,  0.2305],
          ...,
          [ 0.2232,  0.1406,  0.1772,  ...,  0.0548,  0.0576,  0.1992],
          [ 0.2063,  0.1480,  0.1588,  ...,  0.0703, -0.0132,  0.1781],
          [ 0.2034,  0.1735,  0.2058,  ...,  0.1362,  0.0304,  0.1573]]],


        [[[ 0.3870,  0.3575,  0.4814,  ...,  0.6260,  0.6921,  0.5607],
          [ 0.3767,  0.340

tensor([[[[-0.0130, -0.1545, -0.2052,  ...,  0.1759,  0.2443,  0.4117],
          [-0.0713, -0.2203, -0.2797,  ...,  0.1249,  0.2790,  0.4519],
          [-0.1644, -0.3296, -0.3559,  ...,  0.0574,  0.2397,  0.4820],
          ...,
          [-0.3602, -0.5005, -0.5087,  ..., -0.0335,  0.0961,  0.2271],
          [-0.2930, -0.4492, -0.4453,  ..., -0.0146,  0.1548,  0.2727],
          [-0.3015, -0.4188, -0.3768,  ...,  0.0279,  0.1469,  0.3001]]],


        [[[-0.1048, -0.1099, -0.1411,  ...,  0.0385,  0.0528,  0.0324],
          [-0.1043, -0.0224, -0.1193,  ...,  0.1217,  0.0636, -0.0318],
          [ 0.0434, -0.0764, -0.1075,  ...,  0.0568,  0.0177, -0.0481],
          ...,
          [ 0.3950,  0.3756,  0.2451,  ...,  0.1375,  0.0665,  0.1397],
          [ 0.1046,  0.0184,  0.0219,  ...,  0.0217, -0.0449, -0.1673],
          [-0.0329, -0.1424, -0.2413,  ..., -0.2224, -0.1661, -0.1518]]],


        [[[-0.0772, -0.1741, -0.2408,  ...,  0.1620,  0.2786,  0.3658],
          [-0.2478, -0.267

In [76]:
print(batch.shape) # 96 pieces of data, 1 channel: checking if can perform on entire batch simultaneously


torch.Size([96, 1, 28, 28])


In [77]:
# Checking if convolution can be performed on Fashion MNIST data
for batch, label in train_loader:
    convoluted = torch.nn.Conv2d(1, 1, kernel_size = (14,14), bias = False)(batch)
    print(convoluted.shape) # Input and output channels both refer to RGB channels, either way dimensionality is maintained across batches
    break

torch.Size([256, 1, 15, 15])


In [78]:
print(batch.shape) 

torch.Size([256, 1, 28, 28])


In [93]:
# Applying init to model to initialize all layer weights
def build_convnet():
    # Kernels are iterating across all three dimensions and all batch sizes simultaneously
    model = ConvNet()
    trainer = torch.optim.Adam(model.parameters(), lr = 0.005)
    loss = torch.nn.CrossEntropyLoss()
    return model, trainer, loss

In [94]:
model, optimizer, cost = build_convnet()


In [95]:
def train_model(model: torch.nn.Module, optimizer, cost, train_loader, epochs: int = 5):
    for epoch in range(epochs):
        for data, labels, in train_loader: # Returns data, label tuple
            optimizer.zero_grad()
            
            # Obtaining cross entropy cost
            loss = cost(model(data), labels)
            
            # Displaying for first epoch
            if epoch == 0:
                print("cost: ", loss)
                
            # Resetting gradient
            # Computing gradients
            loss.backward()
            # Displaying cost every 10 iterations
            optimizer.step()
        
        # Printing cost every 10th epoch
        print("cost: ", loss)

In [96]:
train_model(model, optimizer, cost, train_loader, 5) # Accuracy much better

cost:  tensor(2.3248, grad_fn=<NllLossBackward>)
cost:  tensor(2.3129, grad_fn=<NllLossBackward>)
cost:  tensor(2.3081, grad_fn=<NllLossBackward>)
cost:  tensor(2.3037, grad_fn=<NllLossBackward>)
cost:  tensor(2.3098, grad_fn=<NllLossBackward>)
cost:  tensor(2.2957, grad_fn=<NllLossBackward>)
cost:  tensor(2.3080, grad_fn=<NllLossBackward>)
cost:  tensor(2.3088, grad_fn=<NllLossBackward>)
cost:  tensor(2.3038, grad_fn=<NllLossBackward>)
cost:  tensor(2.3031, grad_fn=<NllLossBackward>)
cost:  tensor(2.3078, grad_fn=<NllLossBackward>)
cost:  tensor(2.3052, grad_fn=<NllLossBackward>)
cost:  tensor(2.3098, grad_fn=<NllLossBackward>)
cost:  tensor(2.3013, grad_fn=<NllLossBackward>)
cost:  tensor(2.3075, grad_fn=<NllLossBackward>)
cost:  tensor(2.3050, grad_fn=<NllLossBackward>)
cost:  tensor(2.3058, grad_fn=<NllLossBackward>)
cost:  tensor(2.3014, grad_fn=<NllLossBackward>)
cost:  tensor(2.3041, grad_fn=<NllLossBackward>)
cost:  tensor(2.3055, grad_fn=<NllLossBackward>)
cost:  tensor(2.3044

cost:  tensor(2.3013, grad_fn=<NllLossBackward>)
cost:  tensor(2.3055, grad_fn=<NllLossBackward>)
cost:  tensor(2.3044, grad_fn=<NllLossBackward>)
cost:  tensor(2.3038, grad_fn=<NllLossBackward>)
cost:  tensor(2.3015, grad_fn=<NllLossBackward>)
cost:  tensor(2.3019, grad_fn=<NllLossBackward>)
cost:  tensor(2.3022, grad_fn=<NllLossBackward>)
cost:  tensor(2.3028, grad_fn=<NllLossBackward>)
cost:  tensor(2.3036, grad_fn=<NllLossBackward>)
cost:  tensor(2.3022, grad_fn=<NllLossBackward>)
cost:  tensor(2.3036, grad_fn=<NllLossBackward>)
cost:  tensor(2.3028, grad_fn=<NllLossBackward>)
cost:  tensor(2.3032, grad_fn=<NllLossBackward>)
cost:  tensor(2.3028, grad_fn=<NllLossBackward>)
cost:  tensor(2.3018, grad_fn=<NllLossBackward>)
cost:  tensor(2.3027, grad_fn=<NllLossBackward>)
cost:  tensor(2.3036, grad_fn=<NllLossBackward>)
cost:  tensor(2.3030, grad_fn=<NllLossBackward>)
cost:  tensor(2.3024, grad_fn=<NllLossBackward>)
cost:  tensor(2.3027, grad_fn=<NllLossBackward>)
cost:  tensor(2.3019

Rigorously, a convolution is left shifted instead of right shifted, rendering this a cross correlation. However, since either way our kernels are learned, there is no difference made in the learning pattern - the kernel transposed would be identical and an identical gradient is applied to the transposed kernel

**Feature Map** - The output of a convolutional layers, as it is a set of localized aggregated features in space.

**Receptive Field** - The set of all elements which affect the computation of a feature. A feature with a large receptive field is a function of a large portion of the image and would represent a pattern taking place in a large portion of the image.

In [97]:
torch.tensor([1,2,3,4]).long().type()

'torch.LongTensor'